First, let's import the necessary modules, and select the orbita2d config you want to use.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tempfile

orbita_config = '/Users/pierrerouanet/Dev/orbita2d_control/orbita2d_controller/config/workbench.yaml'

Then, define your roll/pitch trajectory.
Timestep is in ms, and the trajectory is in radians.

In [ ]:
# Sinusoid on pitch
t = np.linspace(0, 10, 1000)
roll = np.zeros(t.shape)
pitch = np.deg2rad(30) * np.sin(2 * np.pi * 0.5 * t)

# Square
t = np.linspace(0, 10, 2000)
roll = np.zeros(t.shape)
pitch = np.zeros(t.shape)
pitch[250:750] = np.deg2rad(30)
pitch[1250:1750] = np.deg2rad(-30)

In [ ]:
plt.plot(roll, label='roll')
plt.plot(pitch, label='pitch')
plt.title('Target trajectory')
plt.legend()

Let's save the trajectory to a file, so we can use it later.
We also use an output file to store the real trajectory's state at each timestep.

In [ ]:
pos = np.vstack((roll, pitch)).T
input_tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.npy')
np.save(input_tmp_file.name, pos)

output_tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.npz')

Now let's run the trajectory player. 

This will play the trajectory, and send the commands to the orbita2d hardware.

It will also save the real trajectory's state (position/velocity/torque) at each timestep (1ms) to the output file.
We also choose to use a raw velocity limit.

In [ ]:
!cargo run --package orbita2d_controller --bin orbita2d_trajectory_player -- \
    -c "{orbita_config}" \
    -i "{input_tmp_file.name}" -o "{output_tmp_file.name}" \
    --raw-velocity-limit 25.0

Let's load the result and plot the recorded position trajectory.

In [ ]:
result = np.load(output_tmp_file.name)

plt.plot(result['position'][:, 0], label='roll')
plt.plot(result['position'][:, 1], label='pitch')
plt.title('Position')
plt.legend()

We can also plot the error (difference between the desired and real trajectory).

In [ ]:
position_error = result['position'] - np.roll(pos, 0, axis=0)

plt.plot(position_error[:, 0], label='roll')
plt.plot(position_error[:, 1], label='pitch')
plt.title('Position error')

Similarly, but with an offset in the trajectory.

In [ ]:
best_shift, best_error = 0, np.inf
for shift in range(1, 100):
    position_error = result['position'] - np.roll(pos, shift, axis=0)
    error = np.linalg.norm(position_error)
    if error < best_error:
        best_error = error
        best_shift = shift


position_error = result['position'] - np.roll(pos, best_shift, axis=0)

plt.plot(position_error[:, 0], label='roll')
plt.plot(position_error[:, 1], label='pitch')
plt.title(f'Position error (shift = {best_shift})')

Now the velocity.

In [ ]:
plt.plot(result['velocity'][:, 0], label='roll')
plt.plot(result['velocity'][:, 1], label='pitch')
plt.title('Velocity')
plt.legend()

And the torque

In [ ]:
plt.plot(result['torque'][:, 0], label='roll')
plt.plot(result['torque'][:, 1], label='pitch')
plt.title('Torque')
plt.legend()